# Configuração

In [2]:

# Tipagem
import typing 
from typing import Any, Tuple

# Auxiliares
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Core
import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow.keras.layers.experimental import preprocessing

# Classe auxliar
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    if isinstance(names, str):
      names = (names,)

    shape = tf.shape(tensor)
    rank = tf.rank(tensor)

    if rank != len(names):
      raise ValueError(f'Rank mismatch:\n'
                       f'    found {rank}: {shape.numpy()}\n'
                       f'    expected {len(names)}: {names}\n')

    for i, name in enumerate(names):
      if isinstance(name, int):
        old_dim = name
      else:
        old_dim = self.shapes.get(name, None)
      new_dim = shape[i]

      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

ModuleNotFoundError: No module named 'numpy'

# Dados

## Carregando os dados

In [11]:
#path_to_zip = tf.keras.utils.get_file('spa-eng.zip', origin='C://Users/ricar/Downloads/spa-eng.zip', extract=True)
path_of_file = pathlib.Path('./data/dataset.txt')

# função para carregar os dados
def load_data(path):
  text = path.read_text(encoding='utf-8')
  
  #pairs vai ter várias listas, com dois elementos em cada uma: a palavra em Inglês e Espanhol
  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  input = [input for target, input in pairs]
  target = [target for target, input in pairs]

  return target, input

target, input = load_data(path_of_file)
print(f'espanhol: {input[-1]}\n')
print(f'inglês: {target[-1]}')


espanhol: Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado.

inglês: If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.


## Estruturando os dados

In [3]:
BUFFER_SIZE = len(input)
BATCH_SIZE = 64

#Vai separar todas as listas (input, targe) em partes menores, de acordo com o tamanho por parte (BUFFER_ZISE)
dataset = tf.data.Dataset.from_tensor_slices((input, target)).shuffle(BUFFER_SIZE)

#Combina elementos do dataset em lotes (listas) de acordo com o tamanho (BATCH_SIZE)
dataset = dataset.batch(BATCH_SIZE)

for example_input_batch, example_target_batch in dataset.take(1):
  print(example_input_batch[:5])
  print()
  print(example_target_batch[:5])
  break


tf.Tensor(
[b'Han mejorado los negocios.' b'Preferir\xc3\xada quedarme.'
 b'\xc2\xbfAcaso viste c\xc3\xb3mo me mir\xc3\xb3?'
 b'Se considera que una persona con un IMC de 25 a 29 padece sobrepeso.'
 b'La erupci\xc3\xb3n volc\xc3\xa1nica amenazaba a la aldea.'], shape=(5,), dtype=string)

tf.Tensor(
[b'Business has improved.' b"I'd rather stay."
 b'Did you see how he looked at me?'
 b'A person with a BMI of 25 to 29 is considered overweight.'
 b'The volcanic eruption threatened the village.'], shape=(5,), dtype=string)


## Pré processamento do texto

In [4]:
#Aqui neste método há várias regras para processar os textos nas línguas distintas
def tf_lower_and_split_punctuation(text):
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)
 
  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

tf_lower_and_split_punctuation('¿Todavía está en casa?').numpy().decode()


'[START] ¿ todavia esta en casa ? [END]'

## Vetorização dos textos

In [5]:
max_vocabulary_size = 5000

#O adapt é usado para fazer um treinamento dos vetores em cima dos dados 

#camada de vetorização do input 
input_text_processor = preprocessing.TextVectorization(standardize=tf_lower_and_split_punctuation, max_tokens=max_vocabulary_size)
input_text_processor.adapt(input)

#camada de vetorização do output 
output_text_processor = preprocessing.TextVectorization(standardize=tf_lower_and_split_punctuation, max_tokens=max_vocabulary_size)
output_text_processor.adapt(target)


## Usando as camadas vetorizadas


In [6]:
#As camadas têm capacidade agora de converter uma lista (ou lote) de strings em textos
#E com o vocabulário, pode-se converter os tokens para textos

print(f'Vocabulário da camada input: {input_text_processor.get_vocabulary()[:10]}')
print(f'Vocabulário da camada output: {output_text_processor.get_vocabulary()[:10]}\n')

example_tokens = input_text_processor(example_input_batch)

print(f'Exemplo de tokens: {example_tokens[:3, :10]}\n')
print(f'Tokens da primeira frase: {example_tokens[0]}\n')

input_vocab = np.array(input_text_processor.get_vocabulary())
tokens = input_vocab[example_tokens[0].numpy()]
first_phrase = ' '.join(tokens)

print(f'Primeira frase "destokenizada": {tokens}\n')
print(f'Primeira frase "destokenizada" e formatada: {first_phrase}')

Vocabulário da camada input: ['', '[UNK]', '[START]', '[END]', '.', 'que', 'de', 'el', 'a', 'no']
Vocabulário da camada output: ['', '[UNK]', '[START]', '[END]', '.', 'the', 'i', 'to', 'you', 'tom']

Exemplo de tokens: [[   2  300 4425   26 1209    4    3    0    0    0]
 [   2 1264  865    4    3    0    0    0    0    0]
 [   2   13  680  782   38   18  661   12    3    0]]

Tokens da primeira frase: [   2  300 4425   26 1209    4    3    0    0    0    0    0    0    0
    0]

Primeira frase "destokenizada": ['[START]' 'han' 'mejorado' 'los' 'negocios' '.' '[END]' '' '' '' '' '' ''
 '' '']

Primeira frase "destokenizada" e formatada: [START] han mejorado los negocios . [END]        


# Modelos

## Codificador

In [7]:
#Variáveis "globais"

embedding_dim = 256 #Dimensão da camada de embedding
units = 1024 #

In [26]:
# Codificador

# Considerações: o uso da classe ShapeChecker() serve para verificar os formatos dos tensores
#                o codificador precisa retornar uma saída codificada e também o seu estado para que seja passado ao decodificador  

# O codificador, simplificando, é uma camada da inteligência de tradução, por isso herda de Layer
class Encoder(tf.keras.layers.Layer):
  def __init__(self, input_vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    self.input_vocab_size = input_vocab_size

    #Camada de embedding para converter tokens em vetores 
    self.embedding = tf.keras.layers.Embedding(self.input_vocab_size, embedding_dim)

    #Essa é a camada que processa os vetores sequencialmente (camada RNN) 
    self.gru = tf.keras.layers.GRU(
      self.enc_units,
      return_sequences=True,
      return_state=True,
      recurrent_initializer='glorot_uniform'
    )
  
  def call(self, tokens, state=None):
    shape_checker = ShapeChecker()
    shape_checker(tokens, ('batch', 's'))

    #Camada de embedding que procura os tokens
    vectors = self.embedding(tokens)
    shape_checker(vectors, ('batch', 's', 'embed_dim'))

    #Camada RNN que processa a sequencia de vetores
    output, state = self.gru(vectors, initial_state=state)
    shape_checker(output, ('batch', 's', 'enc_units'))
    shape_checker(state, ('batch', 'enc_units'))

    return output, state

## Testando o codificador

In [28]:
#Convertendo a entrada (texto) para tokens
example_tokens = input_text_processor(example_input_batch[:32])

#Tenho 64 frases dentro de "example_input_batch"
print(f'formato do lote de entrada (lote de frases): {example_input_batch.shape}')
print(f'algumas frases do lote: {example_input_batch[:2]}\n')

#Tenho 64 lista de tokens, cada lista contendo 15 tokens
print(f'formato do lote de tokens de entrada (lote de tokens): {example_tokens.shape}')
print(f'alguns tokens que estão sendo passados: {example_tokens[:1]}\n')

#Codificando a sequência de tokens de entrada
encoder = Encoder(input_text_processor.vocabulary_size(), embedding_dim, units)
example_enc_output, example_enc_state = encoder(example_tokens)

#Eu tenho uma saída, codificada, com o seguinte shape (x, 15, 1024)
#Onde x é quantidade de frases que passei, 15 é quantidade de tokens e 1024 é a dimensão de cada embedding 
print(f'Saída codificada, shape (batch, s, units): {example_enc_output.shape}')
print(f'Estado de saída, shape (batch, units): {example_enc_state.shape}\n')

#Cada frase vai ter uma lista de tokens que foram vetorizados à embeddings (transformado em uma lista de valores de embeddings)
#Então agora temos 15 listas, referente ao valor dos tokens, e cada lista tem uma lista com 1024 valores de embedding 
print(f'Primeira frase: {example_enc_output[0].shape}')
print(f'Valor de embedding do primeiro token: {example_enc_output[0][0].shape}')

formato do lote de entrada (lote de frases): (64,)
algumas frases do lote: [b'Han mejorado los negocios.' b'Preferir\xc3\xada quedarme.']

formato do lote de tokens de entrada (lote de tokens): (32, 15)
alguns tokens que estão sendo passados: [[   2  300 4425   26 1209    4    3    0    0    0    0    0    0    0
     0]]

Saída codificada, shape (batch, s, units): (32, 15, 1024)
Estado de saída, shape (batch, units): (32, 1024)

Primeira frase: (15, 1024)
Embedding da primeira frase: (1024,)
